In [1]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.1
SparkUI available at http://45df7160f5c6:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.47-d9e1f3a110c8
LOGGING: writing to /home/jovyan/work/repos/sgkit-dev/notebook/plink/hail-20200705-2226-0.2.47-d9e1f3a110c8.log


In [2]:
mt = hl.balding_nichols_model(1, n_samples=10, n_variants=100)
mt = mt.annotate_entries(GT=hl.if_else(hl.rand_unif(0, 1, seed=1) < .9, mt.GT, hl.null('call')))
mt = mt.annotate_cols(s=hl.str(mt.sample_idx))
mt = mt.key_cols_by('s')
mt.describe()

2020-07-05 22:26:22 Hail: INFO: balding_nichols_model: generating genotypes for 1 populations, 10 samples, and 100 variants...


----------------------------------------
Global fields:
    'bn': struct {
        n_populations: int32, 
        n_samples: int32, 
        n_variants: int32, 
        n_partitions: int32, 
        pop_dist: array<int32>, 
        fst: array<float64>, 
        mixture: bool
    }
----------------------------------------
Column fields:
    'sample_idx': int32
    'pop': int32
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'ancestral_af': float64
    'af': array<float64>
----------------------------------------
Entry fields:
    'GT': call
----------------------------------------
Column key: ['s']
Row key: ['locus', 'alleles']
----------------------------------------


In [3]:
hl.export_plink(mt, '/tmp/plink_sim_10s_100v_10pmiss')

2020-07-05 22:26:26 Hail: INFO: Coerced sorted dataset
2020-07-05 22:26:26 Hail: INFO: merging 9 files totalling 300...
2020-07-05 22:26:26 Hail: INFO: while writing:
    /tmp/plink_sim_10s_100v_10pmiss.bed
  merge time: 13.504ms
2020-07-05 22:26:26 Hail: INFO: merging 8 files totalling 2.1K...
2020-07-05 22:26:26 Hail: INFO: while writing:
    /tmp/plink_sim_10s_100v_10pmiss.bim
  merge time: 8.362ms
2020-07-05 22:26:27 Hail: INFO: merging 16 files totalling 130...
2020-07-05 22:26:27 Hail: INFO: while writing:
    /tmp/plink_sim_10s_100v_10pmiss.fam
  merge time: 10.048ms
2020-07-05 22:26:27 Hail: INFO: wrote 100 variants and 10 samples to '/tmp/plink_sim_10s_100v_10pmiss'


In [16]:
mtp = mt.annotate_cols(alt_allele_count=hl.agg.sum(mt.GT.n_alt_alleles()))
mtp.alt_allele_count.collect()

[102, 95, 98, 94, 88, 91, 90, 98, 96, 103]

In [17]:
mtp = mt.annotate_cols(call_rate=hl.agg.mean(hl.is_defined(mt.GT.n_alt_alleles())))
mtp.call_rate.collect()

[0.95, 0.9, 0.91, 0.87, 0.86, 0.83, 0.86, 0.87, 0.92, 0.92]